In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!curl --header "Host: launchpadlibrarian.net" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header "Accept-Language: zh,zh-CN;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6" "https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb" -o "google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb" -L
!apt-get -y install -qq fuse
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
!ls drive > /dev/null

### Setup

In [ ]:
!mkdir src -p && cd src && git clone https://github.com/fastai/fastai.git

In [ ]:
!pip3 install -q bcolz graphviz sklearn_pandas isoweek pandas_summary ipywidgets torch torchvision torchtext

In [0]:
!git config --global user.email 'nikshuang@163.com'
!git config --global user.name 'Alex Huang'
!pip install -q kaggle
!pip install -q -U git+https://github.com/albu/albumentations

In [ ]:
!git clone https://github.com/svishnu88/TGS-SaltIdentification-Open-Solution-fastai.git
!cp TGS-SaltIdentification-Open-Solution-fastai/*.py .
!pip install -q Cython --upgrade
!pip install -q pycocotools --upgrade

In [0]:
import sys
sys.path.append("/content/src/fastai/old") # on windows use \'s instead

from fastai.conv_learner import *
from fastai.dataset import *
from skimage.transform import resize
import gc
from sklearn.model_selection import train_test_split, StratifiedKFold , KFold
from sklearn.metrics import jaccard_similarity_score
from pycocotools import mask as cocomask
from utils import my_eval,intersection_over_union_thresholds,RLenc
from lovasz_losses import *
print(torch.__version__)
torch.backends.cudnn.benchmark=True
torch.cuda.is_available(), torch.backends.cudnn.enabled

0.4.1


(True, True)

In [0]:
PATH = Path('data/salt')
TRN_PATH = PATH/'train'
TEST_PATH = PATH/'test'
DISK_PATH = 'drive/tgs_salt'
os.makedirs(DISK_PATH, exist_ok=True)
os.makedirs(PATH, exist_ok=True)
os.makedirs(TRN_PATH, exist_ok=True)
os.makedirs(TEST_PATH, exist_ok=True)

In [0]:
from albumentations import (
    Resize,
    HorizontalFlip,
    VerticalFlip,
    Compose,
    RandomRotate90,
    RandomSizedCrop,
    OneOf,
    RandomSizedCrop,
    Rotate,
    RandomContrast,
    RandomGamma,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomBrightness
)   

In [ ]:
!wget http://files.fast.ai/models/weights.tgz
!tar xf weights.tgz -C /content/src/fastai/old/fastai/
!ls /content/src/fastai/old/fastai/weights

In [ ]:
!kaggle competitions download -c tgs-salt-identification-challenge -p {PATH}

In [0]:
!cd {TRN_PATH} && unzip -q /content/{PATH}/'train.zip'
!cd {TEST_PATH} && unzip -q /content/{PATH}/'test.zip'

In [0]:
!ls {TRN_PATH}
!ls {TEST_PATH}

images	masks
images


### EDA

In [0]:
depth = pd.read_csv(PATH/'depths.csv')
train = pd.read_csv(PATH/'train.csv')
submission = pd.read_csv(PATH/'sample_submission.csv')

In [0]:
submission.head()

In [0]:
depth.head()

In [0]:
depth.set_index('id').z.to_dict()

In [0]:
depth.z.to_dict()

In [0]:
train.head()

In [0]:
!ls {TRN_PATH}/images | head -10

In [0]:
IMG_ID = '008a50a2ec'

In [0]:
Image.open(TRN_IMG_PATH/f'{IMG_ID}.png').resize((32, 32))

In [0]:
Image.open(TRN_MASK_PATH/f'{IMG_ID}.png').resize((32, 32))

In [0]:
rn34 = torchvision.models.resnet34()

In [0]:
rn34

In [0]:
rn34.layer2

In [0]:
m_base = get_base_model(arch, cut, True)

In [0]:
m_base[4]

In [0]:
pretrained_state_dict = torch.load('/root/.torch/models/resnet34-333f7ec4.pth')

In [0]:
pretrained_state_dict.keys()

In [0]:
arch()

### Dataset

In [0]:
TRN_DN = 'train/images'
MASK_DN = 'train/masks'
TEST_DN = 'test/images'
TRN_PATH = PATH/TRN_DN
MASK_PATH = PATH/MASK_DN

In [0]:
today = datetime.datetime.now().strftime('%Y%m%d')

def save_model_weights(path=DISK_PATH):
    targ_dir = f'{path}/{today}/models'
    os.makedirs(targ_dir, exist_ok=True)
    for f in list(PATH.glob('**/*.h5')):
        shutil.copy('/'.join(f.parts), targ_dir)
        
def load_model_weights(path=PATH, disk_path=DISK_PATH, subdir=None):
    if subdir == None: subdir = today
    os.makedirs(f'{path}/models', exist_ok=True)
    src_dir = f'{disk_path}/{subdir}/models'
    targ_dir = f'{path}/models'
    for f in os.listdir(src_dir):
        shutil.copy(os.path.join(src_dir, f), targ_dir)
        
def save_dataset(path=DISK_PATH):
    targ_dir = f'{path}/{today}'
    os.makedirs(targ_dir, exist_ok=True)
    shutil.copy('test_x.pkl', targ_dir)
    shutil.copy('train_folds.pkl', targ_dir)
    shutil.copy('val_folds.pkl', targ_dir)
    
def load_dataset(path=PATH, disk_path=DISK_PATH, time_stamp=None):
    if time_stamp == None: time_stamp = today
    src_dir = f'{disk_path}/{time_stamp}'
#     assert os.path.exists(src_dir) == True
    shutil.copy(f'{src_dir}/test_x.pkl', '.')
    shutil.copy(f'{src_dir}/train_folds.pkl', '.')
    shutil.copy(f'{src_dir}/val_folds.pkl', '.')

In [0]:
class DepthDataset(Dataset):
    def __init__(self,ds,dpth_dict):
        self.dpth = dpth_dict
        self.ds = ds
        
    def __getitem__(self,i):
        val = self.ds[i]
        return val[0],self.dpth[self.ds.fnames[i].split('/')[-1][:-4]],val[1]
    
class DepthDatasetV2(Dataset):
    def __init__(self,ds,dpth_dict):
        self.dpth = dpth_dict
        self.ds = ds
        
    def __getitem__(self,i):
        val = self.ds[i]
        return val[0],self.dpth[self.ds.fnames[i].name[:-4]],val[1]
      
class MatchedFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        self.y=y
        assert(len(fnames)==len(y))
        super().__init__(fnames, transform, path)
        
    def get_x(self, i): 
        return open_image(os.path.join(self.path, self.fnames[i]))
    
    def get_y(self, i):
        return open_image(os.path.join(str(self.path), str(self.y[i])))

    def get_c(self): return 0
    
class TestFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform,flip, path):
        self.y=y
        self.flip = flip
        super().__init__(fnames, transform, path)
        
    def get_x(self, i): 
        im = open_image(os.path.join(self.path, self.fnames[i]))
        return np.fliplr(im) if self.flip else im
        
    def get_y(self, i):
        im = open_image(os.path.join(str(self.path), str(self.y[i])))
        return np.fliplr(im) if self.flip else im
    def get_c(self): return 0

In [0]:
# trn = pd.read_csv(PATH/'train.csv')
dpth = pd.read_csv(PATH/'depths.csv')
c = dpth.set_index('id')
dpth_dict = c['z'].to_dict()

In [0]:
kf = 5
# kf = 10

In [0]:
x_names = np.array([Path('/'.join([TRN_DN, f])) for f in list(os.listdir(TRN_PATH))])
y_names = np.array([Path('/'.join([MASK_DN, f])) for f in list(os.listdir(TRN_PATH))])
test_x = np.array([Path('/'.join([TEST_DN, f.name])) for f in list((PATH/TEST_DN).iterdir())])
f_name = [o.name for o in x_names]

c = dpth.set_index('id')
dpth_dict = c['z'].to_dict()

kfold = KFold(n_splits=kf, shuffle=True, random_state=42)

train_folds = []
val_folds = []
for idxs in kfold.split(f_name):
    train_folds.append([f_name[idx] for idx in idxs[0]])
    val_folds.append([f_name[idx] for idx in idxs[1]])

with open('train_folds.pkl', 'wb') as fp:
  pickle.dump(train_folds, fp)
with open('val_folds.pkl', 'wb') as fp:
  pickle.dump(val_folds, fp)
with open('test_x.pkl', 'wb') as fp:
  pickle.dump(test_x, fp)

save_dataset()

In [0]:
load_dataset(time_stamp='20181008') # 5 folds
# load_dataset(time_stamp='20181016') # 10 folds

In [0]:
train_folds = pickle.load(open('train_folds.pkl',mode='rb'))
val_folds = pickle.load(open('val_folds.pkl',mode='rb'))
test_x = pickle.load(open('test_x.pkl',mode='rb'))

In [0]:
def open_image(fn):
    """ Opens an image using OpenCV given the file path.

    Arguments:
        fn: the file path of the image

    Returns:
        The image in RGB format as numpy array of floats normalized to range between 0.0 - 1.0
    """
    flags = cv2.IMREAD_UNCHANGED+cv2.IMREAD_ANYDEPTH+cv2.IMREAD_ANYCOLOR
    if not os.path.exists(fn) and not str(fn).startswith("http"):
        raise OSError('No such file or directory: {}'.format(fn))
    elif os.path.isdir(fn) and not str(fn).startswith("http"):
        raise OSError('Is a directory: {}'.format(fn))
#     elif isdicom(fn):
#         slice = pydicom.read_file(fn)
#         if slice.PhotometricInterpretation.startswith('MONOCHROME'):
#             # Make a fake RGB image
#             im = np.stack([slice.pixel_array]*3,-1)
#             return im / ((1 << slice.BitsStored)-1)
#         else:
#             # No support for RGB yet, as it involves various color spaces.
#             # It shouldn't be too difficult to add though, if needed.
#             raise OSError('Unsupported DICOM image with PhotometricInterpretation=={}'.format(slice.PhotometricInterpretation))
    else:
        #res = np.array(Image.open(fn), dtype=np.float32)/255
        #if len(res.shape)==2: res = np.repeat(res[...,None],3,2)
        #return res
        try:
            if str(fn).startswith("http"):
                req = urllib.urlopen(str(fn))
                image = np.asarray(bytearray(req.read()), dtype="uint8")
                im = cv2.imdecode(image, flags).astype(np.float32)/255
            else:
                im = cv2.imread(str(fn), flags).astype(np.float32)/255
            if im is None: raise OSError(f'File not recognized by opencv: {fn}')
            return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        except Exception as e:
            raise OSError('Error handling image at: {}'.format(fn)) from e

### Model

In [0]:
class SCSEModule(nn.Module):
    def __init__(self, ch, re=16):
        super().__init__()
        self.cSE = nn.Sequential(nn.AdaptiveAvgPool2d(1),
                                 nn.Conv2d(ch,ch//re,1),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(ch//re,ch,1),
                                 nn.Sigmoid())
        self.sSE = nn.Sequential(nn.Conv2d(ch,ch,1),
                                 nn.Sigmoid())

    def forward(self, x):
        return x * self.cSE(x) + x * self.sSE(x)


class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()


class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.x_scse = SCSEModule(x_in)
        self.up_scse = SCSEModule(up_in)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(self.up_scse(up_p))
        x_p = self.x_conv(self.x_scse(x_p))
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))


class HyperColumn(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = nn.Conv2d(nf, nf // 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(nf // 16, 1, 1)
        self.bn = nn.BatchNorm2d(nf // 16)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        return self.conv2(self.bn(self.relu(self.conv1(x))))


class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,128)
        self.up2 = UnetBlock(128,128,128)
        self.up3 = UnetBlock(128,64,128)
        self.up4 = UnetBlock(128,64,128)
#         self.up5 = nn.ConvTranspose2d(128, 1, 2, stride=2)
        self.up5 = nn.ConvTranspose2d(128, 16, 2, stride=2)
        self.dc = nn.Conv2d(128, 16, 1)
        self.drop = nn.Dropout2d(p=0.5, inplace=True)
        self.logit = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(80, 1, 1)
        )
        
    def forward(self,img,depth):
        x = F.relu(self.rn(img))
        x1 = self.up1(x, self.sfs[3].features)
        x2 = self.up2(x1, self.sfs[2].features)
        x3 = self.up3(x2, self.sfs[1].features)
        x4 = self.up4(x3, self.sfs[0].features)
#         x = self.up5(x4)
        x5 = self.up5(x4)
        x = torch.cat((x5,
                       F.interpolate(self.dc(x4),scale_factor=2,align_corners=False,mode='bilinear'),
                       F.interpolate(self.dc(x3),scale_factor=4,align_corners=False,mode='bilinear'),
                       F.interpolate(self.dc(x2),scale_factor=8,align_corners=False,mode='bilinear'),
                       F.interpolate(self.dc(x1),scale_factor=16,align_corners=False,mode='bilinear')),1)
        x = self.logit(self.drop(x))
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()


class UnetModel():
    def __init__(self,model,lr_cut,name='unet'):
        self.model,self.name = model,name
        self.lr_cut = lr_cut

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [self.lr_cut]))
        return lgs + [children(self.model)[1:]]

In [0]:
def dechannels(nf):
    return nn.Sequential(
        nn.Conv2d(nf, 32, 1),
        nn.ReLU(inplace=True)
    )
  

class Unet50(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(2048, 1024, 512)
        self.up2 = UnetBlock(512, 512, 256)
        self.up3 = UnetBlock(256, 256, 128)
        self.up4 = UnetBlock(128, 64, 128)
        self.up5 = nn.ConvTranspose2d(128, 1, 2, stride=2)
#         self.up5 = nn.ConvTranspose2d(128, 128, 2, stride=2)
#         self.d1 = dechannels(512)
#         self.d2 = dechannels(256)
#         self.d3 = dechannels(128)
#         self.d4 = dechannels(128)
#         self.d5 = dechannels(128)
#         self.drop = nn.Dropout2d(p=0.5, inplace=True)
#         self.logit = nn.Conv2d(160, 1, 1)
#         self.logit = nn.Sequential(
#             nn.Conv2d(1152, 128, 3, 1, 1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(128, 128, 1)
#         )
        
    def forward(self,img,depth):
        x = F.relu(self.rn(img))
        x1 = self.up1(x, self.sfs[3].features)
        x2 = self.up2(x1, self.sfs[2].features)
        x3 = self.up3(x2, self.sfs[1].features)
        x4 = self.up4(x3, self.sfs[0].features)
        x = self.up5(x4)
#         x5 = self.up5(x4)
# #         x = torch.cat((x5,
# #                        F.interpolate(x4,scale_factor=2,align_corners=False,mode='bilinear'),
# #                        F.interpolate(x3,scale_factor=4,align_corners=False,mode='bilinear'),
# #                        F.interpolate(x2,scale_factor=8,align_corners=False,mode='bilinear'),
# #                        F.interpolate(x1,scale_factor=16,align_corners=False,mode='bilinear')),1)
#         x = torch.cat((self.d5(x5),
#                        F.interpolate(self.d4(x4),scale_factor=2,align_corners=False,mode='bilinear'),
#                        F.interpolate(self.d3(x3),scale_factor=4,align_corners=False,mode='bilinear'),
#                        F.interpolate(self.d2(x2),scale_factor=8,align_corners=False,mode='bilinear'),
#                        F.interpolate(self.d1(x1),scale_factor=16,align_corners=False,mode='bilinear')),1)
#         x = self.logit(self.drop(x))
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [0]:
def get_tgs_model():
#     f = resnet34
    f = resnext50
    cut,lr_cut = model_meta[f]
    m_base = get_base(f,cut)
#     m = to_gpu(Unet34(m_base))
    m = to_gpu(Unet50(m_base))
    models = UnetModel(m,lr_cut)
    learn = ConvLearner(md, models)
    return learn

def get_base(f,cut):
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers)

In [0]:
gc.collect()

800

### Training

In [0]:
load_model_weights(subdir='tags/score_8.41')
# load_model_weights(subdir='20181016')
# load_model_weights(subdir='20181018')

In [0]:
def lovasz_hinge_flat(logits, labels):
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * Variable(signs.data))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.elu(errors_sorted) +1, Variable(grad.data))
    return loss

  
def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss

In [0]:
sz = 128
# sz = 192

aug = Compose([
    OneOf([RandomSizedCrop(min_max_height=(50, 101), height=sz, width=sz, p=0.5),
          Resize(sz, sz, p=0.5)], p=1),
    OneOf([VerticalFlip(p=0.5), RandomRotate90(p=0.5), HorizontalFlip(p=0.5)], p=0.5),
#     OneOf([
#         ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
#         GridDistortion(p=0.5),
#         OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)                  
#         ], p=0.8),
#     RandomContrast(limit=0.1, p=0.5),
    RandomBrightness(limit=0.1, p=0.5),
#     RandomGamma(p=0.5)
])

In [0]:
class Transforms():
    def __init__(self, sz, tfms, normalizer, denorm, crop_type=CropType.CENTER,
                 tfm_y=TfmType.NO, sz_y=None, is_trn=True):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        crop_tfm = crop_fn_lu[crop_type](sz, tfm_y, sz_y)
        self.tfms = tfms
        self.tfms.append(crop_tfm)
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))
        self.channel_order = ChannelOrder(tfm_y)
        self.trn_tfms = aug if is_trn == True else None

#     def __call__(self, im, y=None): return compose(im, y, self.trn_tfms, self.tfms, self.norm)
    def __call__(self, im, y=None): return self.compose(im, y)
    def __repr__(self): return str(self.trn_tfms) if self.trn_tfms else str(self.tfms)
    
    def compose(self, im, y):
        if self.trn_tfms:
            augmented = self.trn_tfms(image=im, mask=y)
            im, y = self.norm(augmented['image'], augmented['mask'])
            return self.channel_order(im, y)
        else:
            for fn in self.tfms:
                im, y =fn(im, y)
            return im if y is None else (im, y)


def image_gen(normalizer, denorm, sz, tfms=None, max_zoom=None, pad=0, crop_type=None,
              tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, scale=None, is_trn=True):
    if tfm_y is None: tfm_y=TfmType.NO
    if tfms is None: tfms=[]
    elif not isinstance(tfms, collections.Iterable): tfms=[tfms]
    if sz_y is None: sz_y = sz
    if scale is None:
        scale = [RandomScale(sz, max_zoom, tfm_y=tfm_y, sz_y=sz_y) if max_zoom is not None
                 else Scale(sz, tfm_y, sz_y=sz_y)]
    elif not is_listy(scale): scale = [scale]
    if pad: scale.append(AddPadding(pad, mode=pad_mode))
    if crop_type!=CropType.GOOGLENET: tfms=scale+tfms
    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y, is_trn=is_trn)
  
  
def tfms_from_stats(stats, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    if aug_tfms is None: aug_tfms=[]
    tfm_norm = Normalize(*stats, tfm_y=tfm_y if norm_y else TfmType.NO) if stats is not None else None
    tfm_denorm = Denormalize(*stats) if stats is not None else None
    val_crop = CropType.CENTER if crop_type in (CropType.RANDOM,CropType.GOOGLENET) else crop_type
    val_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=val_crop,
            tfm_y=tfm_y, sz_y=sz_y, scale=scale, is_trn=False)
    trn_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=crop_type,
            tfm_y=tfm_y, sz_y=sz_y, tfms=aug_tfms, max_zoom=max_zoom, pad_mode=pad_mode, scale=scale, is_trn=True)
    return trn_tfm, val_tfm
  
  
def tfms_from_model(f_model, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    stats = inception_stats if f_model in inception_models else imagenet_stats
    return tfms_from_stats(stats, sz, aug_tfms, max_zoom=max_zoom, pad=pad, crop_type=crop_type,
                           tfm_y=tfm_y, sz_y=sz_y, pad_mode=pad_mode, norm_y=norm_y, scale=scale)

### Training: resnext50 + scSE + hypercolumns

In [0]:
model = 'resnext50_128_scse_hcol'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=4, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = nn.BCEWithLogitsLoss()
        if os.path.exists(pa):
            learn.load(pa)
        learn.fit(lrs/2,1, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])
        
        del learn
        del md
        gc.collect()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_hcol_0


epoch      trn_loss   val_loss   my_eval    
    0      0.296024   0.229948   0.6635    
    1      0.226501   0.167776   0.7175    
    2      0.197844   0.174305   0.70425   
    3      0.178601   0.157858   0.734     
    4      0.159284   0.141243   0.758375  
    5      0.156893   0.144844   0.7555    
    6      0.139704   0.14005    0.76825   
    7      0.129483   0.135873   0.782125  
    8      0.117466   0.133637   0.77575   
    9      0.107742   0.148499   0.7775    

0.7821250000000001
fold_id1
5_fold_resnext50_128_scse_hcol_1


epoch      trn_loss   val_loss   my_eval    
    0      0.298795   0.330592   0.577375  
    1      0.232322   0.154958   0.684125  
    2      0.202833   0.152708   0.708     
    3      0.182856   0.165284   0.688875  
    4      0.160326   0.142061   0.73875   
    5      0.149245   0.133409   0.737375  
    6      0.142726   0.129582   0.747375  
    7      0.136257   0.128909   0.754375  
    8      0.121961   0.14172    0.746     
    9      0.111444   0.138969   0.755375  

0.7553750000000001
fold_id2
5_fold_resnext50_128_scse_hcol_2


epoch      trn_loss   val_loss   my_eval    
    0      0.302976   0.230204   0.6545    
    1      0.236865   0.176783   0.70275   
    2      0.200898   0.158097   0.706625  
    3      0.182162   0.147802   0.752     
    4      0.1738     0.146175   0.73025   
    5      0.157935   0.135695   0.758875  
    6      0.148242   0.143257   0.76125   
    7      0.135907   0.130015   0.768625  
    8      0.125547   0.12942    0.761     
    9      0.114384   0.135982   0.76275   

0.7686250000000001
fold_id3
5_fold_resnext50_128_scse_hcol_3


epoch      trn_loss   val_loss   my_eval    
    0      0.298731   0.179404   0.649875  
    1      0.236641   0.150297   0.6965    
    2      0.208887   0.130067   0.725375  
    3      0.187371   0.116859   0.7545    
    4      0.170393   0.127697   0.749     
    5      0.160323   0.117837   0.75725   
    6      0.142183   0.105238   0.78125   
    7      0.135057   0.109958   0.776875  
    8      0.120477   0.110117   0.794     
    9      0.113762   0.101085   0.7865    

0.794
fold_id4
5_fold_resnext50_128_scse_hcol_4


epoch      trn_loss   val_loss   my_eval    
    0      0.302227   0.204683   0.65575   
    1      0.229877   0.183582   0.71175   
    2      0.199228   0.171225   0.68475   
    3      0.192234   0.1548     0.734625  
    4      0.167416   0.140409   0.74525   
    5      0.149521   0.126696   0.743125  
    6      0.136639   0.126258   0.774875  
    7      0.125201   0.119398   0.774875  
    8      0.120503   0.138388   0.762875  
    9      0.111816   0.117212   0.782875  

0.782875


In [0]:
model = 'resnext50_128_scse_hcol'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
        learn.fit(lrs/2,2, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
        learn.fit(lrs/3,1, wds=wd, cycle_len=20,use_clr=(20,16),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])
        
        del learn
        del md
        gc.collect()

save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_hcol_0


epoch      trn_loss   val_loss   my_eval    
    0      0.977655   0.9829     0.76025   
    1      0.984492   1.024706   0.763625  
    2      0.94598    0.968942   0.768125  
    3      0.91825    0.867097   0.782625  
    4      0.842798   0.835175   0.7835    
    5      0.811193   0.819348   0.79875   
    6      0.784356   0.763691   0.8005    
    7      0.763975   0.773445   0.800375  
    8      0.707037   0.741929   0.79875   
    9      0.676857   0.763426   0.80925   
    10     0.686296   0.84298    0.7955    
    11     0.778831   0.926706   0.788625  
    12     0.823366   0.929922   0.78175   
    13     0.846463   0.840977   0.768375  
    14     0.80643    0.766691   0.79525   
    15     0.785104   0.772868   0.793375  
    16     0.771523   0.761224   0.803625  
    17     0.73849    0.711444   0.80725   
    18     0.698676   0.748108   0.807125  
    19     0.664383   0.732889   0.804625  



epoch      trn_loss   val_loss   my_eval    
    0      0.642692   0.774102   0.79425   
    1      0.670723   0.791728   0.797125  
    2      0.701122   0.799743   0.794125  
    3      0.695085   0.809558   0.8015    
    4      0.709821   0.699042   0.801     
    5      0.686103   0.869248   0.78125   
    6      0.668086   0.807663   0.79025   
    7      0.708722   0.761571   0.786875  
    8      0.695549   0.760038   0.809125  
    9      0.710776   0.817635   0.794625  
    10     0.667159   0.799424   0.79725   
    11     0.674622   0.791136   0.797375  
    12     0.653431   0.732944   0.809125  
    13     0.618604   0.733997   0.803     
    14     0.614818   0.781419   0.8115    
    15     0.592244   0.753463   0.8135    
    16     0.593055   0.76313    0.818875  
    17     0.591503   0.800476   0.791875  
    18     0.592983   0.749201   0.816125  
    19     0.586945   0.764278   0.814375  

0.818875
fold_id1
5_fold_resnext50_128_scse_hcol_1


epoch      trn_loss   val_loss   my_eval    
    0      0.871871   1.046633   0.740375  
    1      0.953284   1.179364   0.718875  
    2      0.927234   0.987735   0.73175   
    3      0.891905   0.919984   0.759875  
    4      0.824233   0.970662   0.76025   
    5      0.779335   0.885995   0.766125  
    6      0.750891   0.903214   0.774125  
    7      0.726601   0.843466   0.783625  
    8      0.684857   0.833532   0.789125  
    9      0.668015   0.820718   0.791125  
    10     0.638963   0.86629    0.77425   
    11     0.764584   1.057496   0.69975   
    12     0.819989   1.005252   0.747375  
    13     0.824506   0.927726   0.76375   
    14     0.803146   0.867914   0.77575   
    15     0.80264    0.916781   0.7715    
    16     0.771997   0.827193   0.786625  
    17     0.727303   0.858733   0.788625  
    18     0.686257   0.846492   0.779625  
    19     0.661692   0.800186   0.79375   



epoch      trn_loss   val_loss   my_eval    
    0      0.680326   0.828174   0.7895    
    1      0.68143    0.84101    0.775875  
    2      0.716512   0.921755   0.780375  
    3      0.716995   0.865955   0.780875  
    4      0.733966   0.841387   0.754875  
    5      0.726975   0.876049   0.77775   
    6      0.702523   0.901705   0.77425   
    7      0.672928   0.878522   0.7805    
    8      0.668419   0.859559   0.7695    
    9      0.645093   0.857982   0.788     
    10     0.646567   0.948428   0.7555    
    11     0.659159   0.892936   0.783125  
    12     0.626061   0.893832   0.787125  
    13     0.633051   0.822176   0.787     
    14     0.60531    0.902848   0.788625  
    15     0.598807   0.832705   0.7875    
    16     0.571184   0.847312   0.787625  
    17     0.563408   0.871148   0.795625  
    18     0.571028   0.850332   0.791875  
    19     0.538525   0.868956   0.798375  

0.7983750000000001
fold_id2
5_fold_resnext50_128_scse_hcol_2


...

epoch      trn_loss   val_loss   my_eval    
    0      0.671766   0.759295   0.791375  
    1      0.731133   0.811743   0.789875  
    2      0.763409   0.791452   0.789625  
    3      0.774678   0.789636   0.792     
    4      0.758107   0.77808    0.772     
    5      0.735919   0.742705   0.796625  
    6      0.720245   0.771582   0.782125  
    7      0.710452   0.743036   0.7825    
    8      0.710112   0.777343   0.790875  
    9      0.736112   0.743986   0.7875    
    10     0.70377    0.773617   0.7935    
    11     0.66605    0.770097   0.803     
    12     0.662434   0.712938   0.808625  
    13     0.652016   0.743833   0.807625  
    14     0.649992   0.737581   0.811125  
    15     0.638489   0.707261   0.81575   
    16     0.596832   0.710357   0.818     
    17     0.595883   0.704559   0.804625  
    18     0.572071   0.705111   0.82025   
    19     0.563588   0.69612    0.82325   

0.8232499999999999
fold_id3
5_fold_resnext50_128_scse_hcol_3


epoch      trn_loss   val_loss   my_eval    
    0      0.910043   0.931082   0.774375  
    1      0.962166   0.841506   0.7835    
    2      0.966172   0.884748   0.76575   
    3      0.919205   0.797827   0.792     
    4      0.887789   0.838587   0.782125  
    5      0.833859   0.758443   0.79875   
    6      0.787326   0.748243   0.7995    
    7      0.768268   0.734664   0.80275   
    8      0.721092   0.701125   0.81575   
    9      0.678771   0.707262   0.817625  
    10     0.686963   0.761993   0.807     
    11     0.797779   0.845824   0.78325   
    12     0.836203   0.781706   0.79575   
    13     0.807488   0.871408   0.7955    
    14     0.786293   0.773449   0.801375  
    15     0.779008   0.795548   0.800875  
    16     0.756668   0.742518   0.80525   
    17     0.705088   0.703899   0.81325   
    18     0.677209   0.693749   0.819125  
    19     0.639317   0.679222   0.82      



epoch      trn_loss   val_loss   my_eval    
    0      0.653376   0.719673   0.812375  
    1      0.701078   0.751476   0.80475   
    2      0.728884   0.854194   0.79975   
    3      0.752242   0.724502   0.8095    
    4      0.704703   0.742548   0.815     
    5      0.716587   0.734281   0.811875  
    6      0.683265   0.743123   0.806875  
    7      0.678836   0.815633   0.793625  
    8      0.689473   0.700409   0.81575   
    9      0.665838   0.669478   0.82075   
    10     0.633243   0.730008   0.816125  
    11     0.655839   0.701937   0.816125  
    12     0.632317   0.649175   0.82675   
    13     0.598759   0.674421   0.83225   
    14     0.596253   0.666588   0.829875  
    15     0.586747   0.670523   0.827125  
    16     0.567371   0.686732   0.824375  
    17     0.56206    0.673798   0.8225    
    18     0.545179   0.67912    0.82925   
    19     0.526347   0.698485   0.827     

0.8322499999999999
fold_id4
5_fold_resnext50_128_scse_hcol_4


epoch      trn_loss   val_loss   my_eval    
    0      0.92871    0.959824   0.784125  
    1      0.949663   0.872785   0.776375  
    2      0.909849   0.817117   0.790375  
    3      0.896153   0.804488   0.794125  
    4      0.83174    0.835692   0.79425   
    5      0.814119   0.780774   0.797125  
    6      0.755571   0.759111   0.801875  
    7      0.729774   0.78065    0.808     
    8      0.674973   0.741045   0.813375  
    9      0.660971   0.72706    0.8115    
    10     0.689892   0.819716   0.786375  
    11     0.789879   0.855918   0.789375  
    12     0.783312   0.796657   0.79      
    13     0.781245   0.913932   0.78375   
    14     0.787869   0.830769   0.782625  
    15     0.799666   0.75475    0.805375  
    16     0.750443   0.712465   0.81      
    17     0.708587   0.7156     0.812875  
    18     0.680404   0.758386   0.805625  
    19     0.647      0.703068   0.813125  



epoch      trn_loss   val_loss   my_eval    
    0      0.625973   0.726331   0.820375  
    1      0.670526   0.805685   0.80375   
    2      0.701503   0.781464   0.788875  
    3      0.69349    0.791473   0.801625  
    4      0.682512   0.745718   0.81325   
    5      0.69381    0.70612    0.806625  
    6      0.665137   0.696765   0.816375  
    7      0.641031   0.729714   0.8195    
    8      0.632783   0.706463   0.80375   
    9      0.634154   0.687877   0.81325   
    10     0.626251   0.690043   0.809375  
    11     0.591347   0.709688   0.815125  
    12     0.575432   0.702248   0.816625  
    13     0.587349   0.694433   0.81625   
    14     0.573425   0.694701   0.820375  
    15     0.584016   0.66213    0.8265    
    16     0.547527   0.703967   0.824625  
    17     0.548586   0.674334   0.82825   
    18     0.534016   0.680121   0.829125  
    19     0.507791   0.67429    0.823375  

0.8291249999999999


In [0]:
model = 'resnext50_128_scse_hcol'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
#         learn.fit(lrs/4,1, wds=wd, cycle_len=20,use_clr=(20,16),best_save_name=pa)
        learn.fit(lrs/4,1, wds=wd, cycle_len=40,use_clr=(20,32),best_save_name=pa)
  
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])
        
        del learn
        del md
        gc.collect()

save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_hcol_0


epoch      trn_loss   val_loss   my_eval    
    0      0.580299   0.837661   0.80625   
    1      0.628745   0.803514   0.798375  
    2      0.619702   0.796008   0.80575   
    3      0.628366   0.750468   0.81175   
    4      0.623308   0.748861   0.802     
    5      0.650031   0.795647   0.803     
    6      0.646449   0.763365   0.81125   
    7      0.636074   0.781185   0.811375  
    8      0.623231   0.742726   0.809875  
    9      0.602579   0.753169   0.813375  
    10     0.585192   0.776042   0.817375  
    11     0.576949   0.790247   0.8175    
    12     0.562295   0.796752   0.813875  
    13     0.56244    0.795689   0.788875  
    14     0.575644   0.806945   0.8145    
    15     0.58317    0.765245   0.81525   
    16     0.560825   0.782946   0.805875  
    17     0.562333   0.797728   0.79975   
    18     0.562816   0.761856   0.816375  
    19     0.553854   0.775206   0.8135    
    20     0.546581   0.714756   0.82125   
    21     0.541287   0.770479 

epoch      trn_loss   val_loss   my_eval    
    0      0.565228   0.871326   0.785875  
    1      0.601723   0.882529   0.79425   
    2      0.642919   0.897614   0.78075   
    3      0.637186   0.878214   0.777875  
    4      0.609784   0.895229   0.79075   
    5      0.609371   0.926091   0.7865    
    6      0.601144   0.836737   0.788125  
    7      0.630633   0.871732   0.7835    
    8      0.611182   0.890666   0.7895    
    9      0.592919   0.831647   0.80075   
    10     0.567077   0.936684   0.786875  
    11     0.553323   0.966901   0.78875   
    12     0.559024   0.944055   0.77      
    13     0.590418   0.850013   0.796     
    14     0.573935   0.856166   0.7945    
    15     0.567616   0.860534   0.79075   
    16     0.569173   0.960924   0.781     
    17     0.558296   0.893143   0.791     
    18     0.542129   0.934519   0.794125  
    19     0.547132   0.855531   0.80075   
    20     0.53702    0.854191   0.796875  
    21     0.538296   0.912877 

epoch      trn_loss   val_loss   my_eval    
    0      0.577075   0.786943   0.80825   
    1      0.609053   0.768417   0.804375  
    2      0.656391   0.73859    0.812125  
    3      0.652841   0.757531   0.797     
    4      0.655581   0.736351   0.809875  
    5      0.623059   0.69868    0.812375  
    6      0.627167   0.77681    0.7915    
    7      0.623288   0.750636   0.802125  
    8      0.620909   0.76361    0.807125  
    9      0.628333   0.755546   0.793625  
    10     0.63122    0.744164   0.79475   
    11     0.63828    0.720994   0.8095    
    12     0.607952   0.712268   0.804125  
    13     0.588277   0.729939   0.80725   
    14     0.585946   0.705454   0.8255    
    15     0.58386    0.691177   0.81425   
    16     0.561116   0.713686   0.812625  
    17     0.553486   0.771892   0.818875  
    18     0.553116   0.707916   0.817875  
    19     0.565598   0.728426   0.815625  
    20     0.554347   0.766951   0.819625  
    21     0.536056   0.723741 

epoch      trn_loss   val_loss   my_eval    
    0      0.575313   0.695703   0.823625  
    1      0.624102   0.780891   0.813625  
    2      0.648184   0.697116   0.81825   
    3      0.651478   0.729888   0.80325   
    4      0.633732   0.689429   0.816875  
    5      0.6391     0.707463   0.821875  
    6      0.608808   0.734975   0.821     
    7      0.583592   0.727842   0.8085    
    8      0.607585   0.754664   0.806125  
    9      0.602211   0.745517   0.82925   
    10     0.58275    0.690118   0.826     
    11     0.577038   0.690472   0.816875  
    12     0.585021   0.703931   0.820875  
    13     0.573149   0.737506   0.822875  
    14     0.593045   0.701834   0.823125  
    15     0.576374   0.722945   0.8195    
    16     0.578671   0.718012   0.824125  
    17     0.576396   0.737476   0.825     
    18     0.562349   0.740334   0.82075   
    19     0.562596   0.703988   0.83375   
    20     0.545302   0.742709   0.824875  
    21     0.536242   0.730253 

epoch      trn_loss   val_loss   my_eval    
    0      0.546006   0.717454   0.8235    
    1      0.563938   0.751986   0.812375  
    2      0.598719   0.75113    0.815     
    3      0.588652   0.798206   0.822     
    4      0.599703   0.805366   0.8095    
    5      0.591598   0.734947   0.82075   
    6      0.596523   0.733508   0.811375  
    7      0.597403   0.742025   0.816125  
    8      0.588319   0.729897   0.80425   
    9      0.57466    0.743337   0.808375  
    10     0.590704   0.786838   0.8205    
    11     0.557619   0.687391   0.81425   
    12     0.539545   0.704461   0.82875   
    13     0.543599   0.810272   0.820625  
    14     0.549854   0.70434    0.819     
    15     0.541127   0.726442   0.820625  
    16     0.547006   0.710707   0.805625  
    17     0.556314   0.704044   0.825     
    18     0.533182   0.687741   0.81375   
    19     0.511729   0.7148     0.834     
    20     0.509799   0.690751   0.8375    
    21     0.496054   0.709561 

In [0]:
model = 'resnext50_128_scse_hcol'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
#         learn.fit(lrs/4,1, wds=wd, cycle_len=20,use_clr=(20,16),best_save_name=pa)
        learn.fit(lrs/5,1, wds=wd, cycle_len=20,use_clr=(20,16),best_save_name=pa)
  
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])
        
        del learn
        del md
        gc.collect()

# save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_hcol_0


epoch      trn_loss   val_loss   my_eval    
    0      0.464316   0.82814    0.816375  
    1      0.503966   0.973039   0.808125  
    2      0.524512   0.875324   0.802125  
    3      0.552585   0.802356   0.8155    
    4      0.524045   0.898251   0.8165    
    5      0.519004   0.884085   0.815     
    6      0.512701   0.761209   0.816625  
    7      0.520394   0.781913   0.80575   
    8      0.501048   0.748156   0.815875  
    9      0.504658   0.800892   0.808875  
    10     0.491141   0.764409   0.813375  
    11     0.480288   0.845715   0.823     
    12     0.482994   0.864056   0.817125  
    13     0.474893   0.844127   0.818     
    14     0.475831   0.82016    0.822375  
    15     0.462766   0.80165    0.816     
    16     0.459729   0.801812   0.822625  
    17     0.452857   0.825043   0.821125  
    18     0.432667   0.798102   0.819125  
    19     0.448399   0.799831   0.82425   

0.8242499999999999
fold_id1
5_fold_resnext50_128_scse_hcol_1


epoch      trn_loss   val_loss   my_eval    
    0      0.460871   0.923814   0.807875  
    1      0.463133   0.902192   0.804     
    2      0.487417   0.895001   0.79675   
    3      0.49072    0.918243   0.7825    
    4      0.51323    0.852118   0.80025   
    5      0.496024   0.872492   0.789875  
    6      0.506847   0.856714   0.802125  
    7      0.506169   0.917013   0.785     
    8      0.480601   0.872236   0.7935    
    9      0.478031   0.859447   0.79525   
    10     0.465379   0.893305   0.79625   
    11     0.455413   0.920751   0.802125  
    12     0.450254   0.921361   0.797125  
    13     0.437148   0.914842   0.802     
    14     0.445062   0.855681   0.797125  
    15     0.438345   0.867074   0.79825   
    16     0.445464   0.852298   0.80075   
    17     0.439749   0.874568   0.79525   
    18     0.434257   0.861378   0.809375  
    19     0.41873    0.868665   0.806125  

0.809375
fold_id2
5_fold_resnext50_128_scse_hcol_2


epoch      trn_loss   val_loss   my_eval    
    0      0.46378    0.7404     0.81825   
    1      0.523088   0.696791   0.81175   
    2      0.561582   0.741509   0.806625  
    3      0.559633   0.743049   0.79175   
 70%|███████   | 35/50 [00:57<00:22,  1.49s/it, loss=0.537]

KeyboardInterrupt: ignored

In [0]:
save_model_weights()

In [0]:
bst_acc,np.mean(bst_acc)#With 128

([0.823875, 0.80875, 0.828, 0.834375, 0.8375], 0.8265)

In [0]:
gc.collect()

### Training: resnext50 + scSE

In [0]:
model = 'resnext50_128_scse'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=4, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = nn.BCEWithLogitsLoss()
        if os.path.exists(pa):
            learn.load(pa)
        learn.fit(lrs/2,1, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

In [0]:
model = 'resnext50_128_scse'
arch = resnet34
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
#         learn.fit(lrs/3,2, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
        learn.fit(lrs/3,3, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

In [0]:
model = 'resnext50_128_scse'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
#         learn.fit(lrs/3,1, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
        learn.fit(lrs/3,1, wds=wd, cycle_len=20,use_clr=(20,10),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

In [0]:
model = 'resnext50_128_scse'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/100,lr/10,lr])

        learn.unfreeze()        
        learn.crit = lovasz_hinge
        learn.load(pa)
#         learn.fit(lrs/3,1, wds=wd, cycle_len=10,use_clr=(20,8),best_save_name=pa)
        learn.fit(lrs/4,1, wds=wd, cycle_len=20,use_clr=(20,16),best_save_name=pa)
        learn.fit(lrs/5,1, wds=wd, cycle_len=40,use_clr=(20,32),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_0


epoch      trn_loss   val_loss   my_eval    
    0      0.533707   0.76256    0.8175    
    1      0.594565   0.858352   0.798875  
    2      0.601546   0.788414   0.812125  
    3      0.602312   0.763777   0.816625  
    4      0.597194   0.756633   0.816125  
    5      0.56907    0.767038   0.81325   
    6      0.564023   0.746384   0.82175   
    7      0.579532   0.831695   0.81125   
    8      0.551373   0.760372   0.810875  
    9      0.54197    0.845872   0.809625  
    10     0.539457   0.726901   0.813625  
    11     0.528602   0.7233     0.81875   
    12     0.520925   0.772535   0.822875  
    13     0.518783   0.764549   0.8235    
    14     0.513134   0.735043   0.830125  
    15     0.510657   0.727242   0.826625  
    16     0.493573   0.704724   0.82725   
    17     0.481923   0.73212    0.829125  
    18     0.471987   0.698919   0.828     
    19     0.463161   0.711147   0.82725   



epoch      trn_loss   val_loss   my_eval    
    0      0.479967   0.827555   0.821125  
    1      0.506115   0.7792     0.807     
    2      0.521105   0.704616   0.823375  
    3      0.531661   0.759747   0.813625  
    4      0.527358   0.76719    0.8165    
    5      0.599692   0.78576    0.8095    
    6      0.579218   0.723072   0.814375  
    7      0.553373   0.735349   0.81025   
    8      0.522601   0.74302    0.82075   
    9      0.51634    0.725892   0.8255    
    10     0.507244   0.736419   0.810875  
    11     0.495299   0.747552   0.8235    
    12     0.517736   0.709832   0.8205    
    13     0.52503    0.702308   0.82      
    14     0.517949   0.763225   0.822875  
    15     0.481496   0.760829   0.825875  
    16     0.467406   0.75723    0.818     
    17     0.48397    0.719035   0.821125  
    18     0.480835   0.74827    0.81975   
    19     0.464327   0.758697   0.822625  
    20     0.460972   0.714269   0.8275    
    21     0.451309   0.763296 

epoch      trn_loss   val_loss   my_eval    
    0      0.467102   0.833521   0.792125  
    1      0.514805   0.866801   0.770625  
    2      0.512858   0.888006   0.78125   
    3      0.523391   0.872697   0.785625  
    4      0.541148   0.89274    0.7915    
    5      0.534021   0.817045   0.794     
    6      0.511217   0.827669   0.80725   
    7      0.504867   0.870196   0.796375  
    8      0.482823   0.813677   0.80225   
    9      0.470734   0.859631   0.79925   
    10     0.50188    0.805922   0.790625  
    11     0.50007    0.865748   0.790125  
    12     0.490715   0.808949   0.79925   
    13     0.467356   0.822104   0.80175   
    14     0.459628   0.823243   0.8005    
    15     0.442727   0.81513    0.789125  
    16     0.433787   0.801445   0.803375  
    17     0.428531   0.821544   0.79725   
    18     0.427835   0.775848   0.80675   
    19     0.41361    0.812282   0.805     



epoch      trn_loss   val_loss   my_eval    
    0      0.443954   0.821174   0.795625  
    1      0.473285   0.845159   0.792     
    2      0.465905   0.985209   0.785     
    3      0.456353   0.974172   0.786875  
    4      0.467155   0.920761   0.79475   
    5      0.470211   0.900441   0.79725   
    6      0.455301   0.96447    0.792     
    7      0.47351    0.90089    0.78875   
    8      0.463646   0.856061   0.794875  
    9      0.443074   0.860522   0.792     
    10     0.429617   0.824053   0.798375  
    11     0.445916   0.875289   0.796375  
    12     0.453059   0.872382   0.798125  
    13     0.447237   0.872192   0.801125  
    14     0.44465    0.867648   0.800875  
    15     0.447324   0.889119   0.795     
    16     0.449389   0.843531   0.800875  
    17     0.438213   0.823923   0.8075    
    18     0.434376   0.847579   0.7975    
    19     0.443935   0.843054   0.801625  
    20     0.422225   0.849103   0.80025   
    21     0.414883   0.866637 

...

epoch      trn_loss   val_loss   my_eval    
    0      0.443196   0.750643   0.826125  
    1      0.463696   0.795019   0.81775   
    2      0.475888   0.753367   0.819375  
    3      0.478167   0.75828    0.814875  
    4      0.471563   0.778656   0.813625  
    5      0.483763   0.742092   0.81475   
    6      0.479101   0.748993   0.81875   
    7      0.476277   0.780733   0.810875  
    8      0.498361   0.69888    0.82975   
    9      0.486142   0.739697   0.81925   
    10     0.457164   0.801588   0.825375  
    11     0.455916   0.745798   0.815875  
    12     0.457613   0.76608    0.818625  
    13     0.45103    0.773046   0.8205    
    14     0.447433   0.764471   0.813     
    15     0.436356   0.770631   0.82025   
    16     0.440452   0.798791   0.824875  
    17     0.43147    0.753223   0.825125  
    18     0.432604   0.828235   0.8245    
    19     0.435804   0.800897   0.81925   
    20     0.416318   0.782468   0.82325   
    21     0.425427   0.760889 

epoch      trn_loss   val_loss   my_eval    
    0      0.496033   0.709309   0.81325   
    1      0.521143   0.718663   0.82275   
    2      0.54441    0.835224   0.81875   
    3      0.549383   0.731907   0.821625  
    4      0.539594   0.64432    0.82425   
    5      0.534994   0.674456   0.82175   
    6      0.529376   0.71022    0.82975   
    7      0.502961   0.68506    0.824875  
    8      0.498022   0.686693   0.830625  
    9      0.500132   0.725364   0.822125  
    10     0.50281    0.675001   0.831875  
    11     0.512494   0.684257   0.836875  
    12     0.484919   0.696663   0.83475   
    13     0.469144   0.709017   0.837875  
    14     0.469601   0.659997   0.834375  
    15     0.46119    0.692171   0.834125  
    16     0.453164   0.69287    0.83025   
    17     0.443224   0.70075    0.834625  
    18     0.44307    0.694192   0.83      
    19     0.428427   0.728258   0.83575   



epoch      trn_loss   val_loss   my_eval    
    0      0.422124   0.726504   0.8295    
    1      0.462113   0.731205   0.8245    
    2      0.486406   0.728322   0.829125  
    3      0.487896   0.765298   0.828875  
    4      0.484002   0.75814    0.8245    
    5      0.491141   0.697839   0.827125  
    6      0.477768   0.709401   0.822625  
    7      0.483948   0.766132   0.82175   
    8      0.490673   0.695626   0.830375  
    9      0.472208   0.698568   0.81975   
    10     0.466838   0.67741    0.827375  
    11     0.463749   0.797842   0.81275   
    12     0.448588   0.679358   0.830125  
    13     0.469459   0.723771   0.828375  
    14     0.452238   0.672009   0.830875  
    15     0.447767   0.722987   0.830875  
    16     0.43008    0.666037   0.831     
    17     0.430931   0.738625   0.827625  
    18     0.427639   0.6799     0.83275   
    19     0.430004   0.645604   0.82625   
    20     0.440587   0.710382   0.825625  
    21     0.427687   0.758092 

epoch      trn_loss   val_loss   my_eval    
    0      0.47414    0.722221   0.827     
    1      0.491114   0.712578   0.82425   
    2      0.51879    0.73661    0.815875  
    3      0.540288   0.692429   0.826375  
    4      0.534986   0.700633   0.827     
    5      0.518888   0.720056   0.826625  
    6      0.510556   0.819519   0.814     
    7      0.498047   0.740013   0.821125  
    8      0.467538   0.726275   0.821875  
    9      0.465198   0.740374   0.82125   
    10     0.454122   0.729831   0.822125  
    11     0.465724   0.69371    0.827625  
    12     0.459633   0.711312   0.82275   
    13     0.44714    0.709925   0.8275    
    14     0.450566   0.755889   0.81475   
    15     0.433707   0.689775   0.836125  
    16     0.419188   0.687338   0.8345    
    17     0.419773   0.675787   0.834     
    18     0.422193   0.681697   0.83225   
    19     0.414564   0.676884   0.833125  



epoch      trn_loss   val_loss   my_eval    
    0      0.416673   0.743005   0.82225   
    1      0.451756   0.796281   0.822     
    2      0.47234    0.772019   0.824125  
    3      0.479158   0.759053   0.807875  
    4      0.46425    0.769957   0.815375  
    5      0.464162   0.772572   0.815875  
    6      0.460326   0.737922   0.820625  
    7      0.468473   0.750393   0.818375  
    8      0.458967   0.704921   0.831375  
    9      0.439696   0.740977   0.832625  
    10     0.445156   0.723011   0.824875  
    11     0.434864   0.725554   0.833625  
    12     0.422101   0.704498   0.828375  
    13     0.429801   0.699662   0.82975   
    14     0.432414   0.763216   0.81775   
    15     0.423432   0.72516    0.822875  
    16     0.425951   0.732884   0.832625  
    17     0.41282    0.699522   0.830125  
    18     0.40507    0.741123   0.8335    
    19     0.406175   0.729666   0.8355    
    20     0.423782   0.71638    0.82775   
    21     0.407595   0.724523 

In [0]:
model = 'resnext50_128_scse'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/150,lr/20,lr])

        learn.unfreeze()
        learn.bn_freeze(True)
        learn.crit = lovasz_hinge
        learn.load(pa)
        learn.fit(lrs/10,1, wds=wd, cycle_len=20,use_clr=(20,20),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

        if i < kf - 1:
            del learn
            del md
            gc.collect()
        
save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_0


epoch      trn_loss   val_loss   my_eval    
    0      0.352572   0.727342   0.824     
    1      0.369447   0.676443   0.83      
    2      0.387457   0.791503   0.825     
    3      0.351119   0.766687   0.827625  
    4      0.36598    0.731143   0.83275   
    5      0.364853   0.723793   0.8195    
    6      0.381143   0.841033   0.829125  
    7      0.364594   0.680827   0.83075   
    8      0.349329   0.828338   0.817375  
    9      0.35967    0.746246   0.834     
    10     0.355637   0.749717   0.83125   
    11     0.353103   0.816089   0.83225   
    12     0.33925    0.821926   0.83525   
    13     0.347948   0.751935   0.83625   
    14     0.325283   0.842732   0.833625  
    15     0.332459   0.769492   0.83075   
    16     0.329128   0.773248   0.83475   
    17     0.326993   0.837361   0.837     
    18     0.33641    0.736122   0.83775   
    19     0.32058    0.791872   0.8365    

0.8377500000000001
fold_id1
5_fold_resnext50_128_scse_1


epoch      trn_loss   val_loss   my_eval    
    0      0.329473   0.895153   0.791     
    1      0.345151   0.857987   0.802125  
    2      0.346842   0.957962   0.787625  
    3      0.359232   0.914885   0.78725   
    4      0.35904    0.945004   0.79475   
    5      0.364206   0.90916    0.79025   
    6      0.352084   0.907089   0.803125  
    7      0.340385   0.884313   0.79975   
    8      0.331111   0.935045   0.80525   
    9      0.327103   0.912742   0.80725   
    10     0.334112   0.903979   0.799875  
    11     0.339094   0.997254   0.79625   
    12     0.336058   0.903867   0.802     
    13     0.330904   0.945125   0.79575   
    14     0.328544   0.888129   0.798625  
    15     0.322872   0.931454   0.7975    
    16     0.320054   0.917544   0.803375  
    17     0.318902   0.909964   0.8055    
    18     0.318903   0.864422   0.80375   
    19     0.319137   0.889656   0.803875  

0.8072499999999999
fold_id2
5_fold_resnext50_128_scse_2


epoch      trn_loss   val_loss   my_eval    
    0      0.334564   0.907237   0.827     
    1      0.356609   0.77755    0.821125  
    2      0.359052   0.832501   0.83225   
    3      0.365679   0.80743    0.821     
    4      0.361891   0.790572   0.814625  
    5      0.343242   0.863644   0.823625  
    6      0.339013   0.821969   0.828     
    7      0.334319   0.807286   0.818625  
    8      0.327691   0.864193   0.828125  
    9      0.328807   0.806045   0.81325   
    10     0.34561    0.859754   0.82925   
    11     0.334322   0.871514   0.828     
    12     0.320915   0.854894   0.823     
    13     0.30964    0.863667   0.8175    
    14     0.3198     0.776783   0.81925   
    15     0.312285   0.829952   0.83175   
    16     0.315182   0.793502   0.8215    
    17     0.304829   0.833349   0.82575   
    18     0.305454   0.814216   0.833375  
    19     0.309406   0.807178   0.82525   

0.8333750000000001
fold_id3
5_fold_resnext50_128_scse_3


epoch      trn_loss   val_loss   my_eval    
    0      0.303919   0.798046   0.829125  
    1      0.330936   0.707061   0.83625   
    2      0.344996   0.750156   0.829125  
    3      0.344157   0.924753   0.833125  
    4      0.326378   0.760323   0.833875  
    5      0.328699   0.870832   0.825625  
    6      0.331057   0.820294   0.83625   
    7      0.34331    0.716414   0.838625  
    8      0.335372   0.924066   0.83275   
    9      0.323994   0.739209   0.83925   
    10     0.334121   0.761376   0.834     
    11     0.322592   0.740011   0.83975   
    12     0.330691   0.819215   0.837125  
    13     0.313002   0.78467    0.842875  
    14     0.318578   0.795953   0.83725   
    15     0.316326   0.765305   0.842625  
    16     0.310763   0.723363   0.84525   
    17     0.296692   0.825693   0.84075   
    18     0.29445    0.823472   0.842625  
    19     0.297867   0.751366   0.83925   

0.8452500000000001
fold_id4
5_fold_resnext50_128_scse_4


epoch      trn_loss   val_loss   my_eval    
    0      0.32856    0.730436   0.834625  
    1      0.335845   0.899728   0.837125  
    2      0.348538   0.913843   0.8295    
    3      0.33758    0.769679   0.8235    
    4      0.346654   0.802165   0.835625  
    5      0.339364   0.746379   0.83825   
    6      0.343552   0.756106   0.828125  
    7      0.331363   0.792459   0.8335    
    8      0.31865    0.735218   0.836125  
    9      0.325964   0.810198   0.839     
    10     0.311348   0.795061   0.841875  
    11     0.321848   0.720627   0.83475   
    12     0.311088   0.884234   0.8375    
    13     0.304974   0.749801   0.836     
    14     0.305366   0.71613    0.83375   
    15     0.314715   0.693329   0.84125   
    16     0.318737   0.787573   0.83675   
    17     0.304981   0.747158   0.83425   
    18     0.302457   0.767533   0.834     
    19     0.299126   0.759485   0.833625  

0.841875


In [0]:
model = 'resnext50_128_scse'
arch = resnext50
bst_acc=[]
use_clr_min=20
use_clr_div=10
aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]

szs = [(128,64)]
for sz,bs in szs:
    print([sz,bs])
    for i in range(kf) :
        print(f'fold_id{i}')
        
        trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[i]])
        trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[i]])
        val_x = [f'{TRN_DN}/{o}' for o in val_folds[i]]
        val_y = [f'{MASK_DN}/{o}' for o in val_folds[i]]
        
        tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
        denorm = md.trn_ds.denorm
        md.trn_dl.dataset = DepthDataset(md.trn_ds,dpth_dict)
        md.val_dl.dataset = DepthDataset(md.val_ds,dpth_dict)
        md.test_dl.dataset = DepthDataset(md.test_ds,dpth_dict)
        learn = get_tgs_model() 
        learn.opt_fn = optim.Adam
        learn.metrics=[my_eval]
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)

        lr=1e-2
        wd=1e-7
        lrs = np.array([lr/150,lr/20,lr])

        learn.unfreeze()
        learn.bn_freeze(True)
        learn.crit = lovasz_hinge
        learn.load(pa)
        learn.fit(lrs/10,1, wds=wd, cycle_len=20,use_clr=(20,20),best_save_name=pa)
               
        learn.load(pa)        
        #Calcuating mean iou score
        v_targ = md.val_ds.ds[:][1]
        v_preds = np.zeros((len(v_targ),sz,sz))     
        v_pred = learn.predict()
        v_pred = to_np(torch.sigmoid(torch.from_numpy(v_pred)))
        p = ((v_pred)>0.5).astype(np.uint8)
        bst_acc.append(intersection_over_union_thresholds(v_targ,p))
        print(bst_acc[-1])

        if i < kf - 1:
            del learn
            del md
            gc.collect()
        
save_model_weights()

[128, 64]
fold_id0
5_fold_resnext50_128_scse_0


epoch      trn_loss   val_loss   my_eval    
    0      0.352473   0.759049   0.829625  
    1      0.363571   0.880663   0.826     
    2      0.35344    0.807012   0.824875  
    3      0.344199   0.811331   0.82425   
    4      0.341503   0.780056   0.826375  
    5      0.349747   0.841056   0.820375  
    6      0.350928   0.883612   0.823375  
    7      0.346245   0.738591   0.819875  
    8      0.339464   0.790437   0.826375  
    9      0.339998   0.844214   0.826875  
    10     0.347131   0.780476   0.83      
    11     0.341404   0.798782   0.829     
    12     0.339125   0.789289   0.829875  
    13     0.334993   0.868688   0.827625  
    14     0.328118   0.880147   0.83275   
    15     0.306373   0.860532   0.831625  
    16     0.304504   0.829747   0.830875  
    17     0.300264   0.837326   0.832625  
    18     0.317846   0.774235   0.836375  
    19     0.310489   0.787925   0.833875  

0.8363749999999999
fold_id1
5_fold_resnext50_128_scse_1


epoch      trn_loss   val_loss   my_eval    
    0      0.327861   0.839905   0.7995    
    1      0.332155   0.878513   0.80625   
    2      0.327904   0.842675   0.78325   
    3      0.323279   1.045407   0.800125  
    4      0.34445    0.838635   0.794375  
    5      0.336768   0.906733   0.8       
    6      0.325074   0.921411   0.804625  
    7      0.331317   0.867638   0.802125  
    8      0.336123   0.883447   0.796875  
    9      0.331685   0.851472   0.80425   
    10     0.315916   0.857878   0.790625  
    11     0.317591   1.001778   0.802125  
    12     0.322167   0.868535   0.79975   
    13     0.324085   0.90506    0.800625  
    14     0.309266   0.923913   0.795375  
    15     0.311752   0.973415   0.79975   
    16     0.300871   0.953171   0.8045    
    17     0.290202   0.962913   0.800125  
    18     0.297091   0.981357   0.80025   
    19     0.299129   0.941654   0.802625  

0.80625
fold_id2
5_fold_resnext50_128_scse_2


epoch      trn_loss   val_loss   my_eval    
    0      0.337081   0.823228   0.83475   
    1      0.344549   0.913375   0.831625  
    2      0.329596   0.828047   0.831125  
    3      0.325009   0.777783   0.8285    
    4      0.33951    0.821626   0.808375  
    5      0.338317   0.940252   0.820875  
    6      0.330713   0.791693   0.83      
    7      0.332787   0.776885   0.82775   
    8      0.332073   0.806915   0.82675   
    9      0.329484   0.799763   0.817375  
    10     0.329214   0.797196   0.82475   
    11     0.31536    0.82193    0.832625  
    12     0.321162   0.843901   0.827375  
    13     0.310198   0.90338    0.828375  
    14     0.292332   0.859527   0.8295    
    15     0.291901   0.843471   0.8235    
    16     0.296143   0.85044    0.819875  
    17     0.302732   0.847719   0.827     
    18     0.303203   0.819016   0.827875  
    19     0.303959   0.815134   0.8255    

0.8347500000000001
fold_id3
5_fold_resnext50_128_scse_3


epoch      trn_loss   val_loss   my_eval    
    0      0.316421   0.866826   0.83525   
    1      0.337529   0.992894   0.828     
    2      0.355061   0.758108   0.83175   
    3      0.341312   0.793649   0.82925   
    4      0.3637     0.741804   0.83575   
    5      0.330458   0.743388   0.839375  
    6      0.334183   0.73421    0.841375  
    7      0.30901    0.724088   0.83675   
    8      0.322369   0.710755   0.830625  
    9      0.312626   0.907613   0.841     
    10     0.314347   0.75424    0.838875  
    11     0.287641   0.762639   0.836125  
    12     0.291472   0.738226   0.844875  
    13     0.294669   0.741112   0.841375  
    14     0.294856   0.747115   0.842375  
    15     0.295397   0.75156    0.843125  
    16     0.287345   0.807438   0.841875  
    17     0.295449   0.754707   0.843625  
 82%|████████▏ | 41/50 [00:54<00:10,  1.22s/it, loss=0.29] 

In [0]:
save_model_weights()

In [0]:
load_model_weights(time_stamp='20181010')

In [0]:
bst_acc,np.mean(bst_acc)#With 128

([0.835, 0.8065000000000001, 0.8315, 0.8447499999999999, 0.8440000000000001],
 0.8323500000000001)

### Submit

In [0]:
# sz = 128
# bs = 64
# model = 'resnext50_128_scse'

# # trn_x = np.array([f'{TRN_DN}/{o}' for o in train_folds[0]])
# # trn_y = np.array([f'{MASK_DN}/{o}' for o in train_folds[0]])
# # val_x = [f'{TRN_DN}/{o}' for o in val_folds[0]]
# # val_y = [f'{MASK_DN}/{o}' for o in val_folds[0]]
        
# aug_tfms = [RandomFlip(tfm_y=TfmType.CLASS)]
# tfms = tfms_from_model(arch, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
# datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
# md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)
# # tfms = tfms_from_model(resnet34, sz=sz, pad=0,crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
# # datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms,test=test_x,path=PATH)
# # md = ImageData(PATH, datasets, bs, num_workers=16, classes=None)

# learn = get_tgs_model() 
# learn.opt_fn = optim.Adam
# learn.crit = lovasz_hinge
# learn.metrics=[my_eval]

In [0]:
preds = np.zeros(shape = (18000,sz,sz))
for o in [True,False]:
    md.test_dl.dataset = TestFilesDataset(test_x,test_x,tfms[1],flip=o,path=PATH)
    md.test_dl.dataset = DepthDatasetV2(md.test_dl.dataset,dpth_dict)
    
    for i in tqdm_notebook(range(kf)):
        pa = f'{kf}_fold_{model}_{i}'
        print(pa)
        learn.load(pa)
        pred = learn.predict(is_test=True)
        pred = to_np(torch.sigmoid(torch.from_numpy(pred)))    
        for im_idx,im in enumerate(pred):
                preds[im_idx] += np.fliplr(im) if o else im
        del pred

5_fold_resnext50_128_scse_0
5_fold_resnext50_128_scse_1
5_fold_resnext50_128_scse_2
5_fold_resnext50_128_scse_3
5_fold_resnext50_128_scse_4



5_fold_resnext50_128_scse_0
5_fold_resnext50_128_scse_1
5_fold_resnext50_128_scse_2
5_fold_resnext50_128_scse_3
5_fold_resnext50_128_scse_4



In [0]:
# plt.imshow(((preds[16]/10)>0.5).astype(np.uint8))

In [0]:
p = [cv2.resize(o/10,dsize=(101,101)) for o in preds]
p = [(o>0.5).astype(np.uint8) for o in p]

In [0]:
p[0].shape

(101, 101)

In [0]:
pred_dict = {id_.name[:-4]:RLenc(p[i]) for i,id_ in tqdm_notebook(enumerate(test_x))}
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

In [0]:
!kaggle competitions submit -c tgs-salt-identification-challenge -f submission.csv -m ""

Successfully submitted to TGS Salt Identification Challenge

In [ ]:
!kaggle competitions submissions -c tgs-salt-identification-challenge

In [0]:
!fusermount -u drive